In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import requests
import csv
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
import urllib
import xlwt
wb = xlwt.Workbook()

country_list = ["CASS 2019", "CASS 2018", "CATS 2017", "CATS 2016", "CATS 2015", "CASS 2014", "CASS 2013", "CASS 2012", "CASS 2011", "CASS 2010", "CASS 2008"]
# country_list = ["CASS 2019", "CASS 2018"]

chromeOptions = webdriver.ChromeOptions()
chromeOptions.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome("C:\\Users\\Kumarsur\\Desktop\\python\\chromedriver",chrome_options=chromeOptions, desired_capabilities=chromeOptions.to_capabilities())

driver.maximize_window()
driver.get("https://tabulator.universumglobal.com/ui/default.aspx") 
time.sleep(2)

obj = driver.switch_to.alert
time.sleep(2)
obj.dismiss()

username = driver.find_element_by_id("txtUserName")
password = driver.find_element_by_id("txtPassword")
username.send_keys("pantheon\surendra.kumar")
password.send_keys("Feb@2019")
login_attempt = driver.find_element_by_xpath("/html/body/form/table/tbody/tr[2]/td[2]/table/tbody/tr[8]/td/table/tbody/tr/td/div/a")
login_attempt.submit()

for country_name in country_list:
    try:
        ###survey searching
        country = country_name
        x_path = "//td[contains(text(),'" + country + "')]"
        search_survey = driver.find_element_by_xpath(x_path)
        search_survey.click()
        time.sleep(1)

        ########################### FRAMES ##############################
        iframeQuestionGroups = driver.find_element_by_id('iframeQuestionGroups')
        iframeQuestions = driver.find_element_by_id('iframeQuestions')
        iframeAnalyze = driver.find_element_by_id('iframeAnalyze')
        iframeCategorize = driver.find_element_by_id('iframeCategorize')
        iframeAlternatives = driver.find_element_by_id('iframeAlternatives')
        iframeFilter = driver.find_element_by_id('iframeFilter')


        ############################# Questions Group frame###########################
        driver.switch_to.frame(iframeQuestionGroups)
        driver.implicitly_wait(30)

        if ("Official Target Groups" in driver.page_source):
            driver.find_element_by_xpath("//span[text() = 'Official Target Groups']").click() ##click Official Target Groups            
        elif ("Educational Profile" in driver.page_source):
            driver.find_element_by_xpath("//span[text() = 'Educational Profile']").click()  ##click Educational Profile            

        time.sleep(1)
        driver.switch_to.default_content()

        ############################# Alternatives frame double click to add FILTER ###########################
        actionChains = ActionChains(driver)
        driver.switch_to.frame(iframeAlternatives)
        driver.implicitly_wait(30)
        time.sleep(1)        
        flag = ""

        if ("Student" in driver.page_source): 
            add_filter= driver.find_element_by_xpath("//option[contains(text(), 'Student')]")  ##add student to filter 
            actionChains.double_click(add_filter).perform()
        elif ("Undergraduate" in driver.page_source): 
            add_filter= driver.find_element_by_xpath("//option[contains(text(), 'Undergraduate')]")  ##add Undergraduate to filter
            actionChains.double_click(add_filter).perform()         
        else:
            flag = 'not any filter'
            
        time.sleep(3)
        driver.switch_to.default_content()

        ############################# Questions frame ###########################
        driver.switch_to.frame(iframeQuestions)
        time.sleep(3)
        select_element = Select(driver.find_element_by_id("listQuestions"))
        el = driver.find_element_by_id('listQuestions')
        for option in el.find_elements_by_tag_name('option'):
            select_element.deselect_by_visible_text(option.text)
        for option in el.find_elements_by_tag_name('option'):
            if option.text == '2. Main Field of Study':
                option.click()
                break
            elif option.text == 'Main Field of Study':
                option.click()
                break
            elif option.text == 'Field of study':
                option.click()
                break

        driver.switch_to.default_content() 
        time.sleep(3)

        ############################# Add to Categorize ###########################
        driver.switch_to.frame(iframeCategorize)
        driver.implicitly_wait(30)
        time.sleep(5)
        driver.find_element_by_xpath("//a[@onclick ='parent.AddToList(document.forms[0].listCategorize);']").click()
        time.sleep(3)
        driver.switch_to.default_content()

        ############################# Questions Group frame ###########################
        driver.switch_to.frame(iframeQuestionGroups)
        driver.implicitly_wait(30)
        driver.find_element_by_xpath("//span[text() = 'Rankings']").click()        
        time.sleep(3)
        driver.switch_to.default_content()

        ############################# Add to Analyze ###########################
        driver.switch_to.frame(iframeAnalyze)
        driver.implicitly_wait(30)
        time.sleep(1)
        driver.find_element_by_xpath("//a[@onclick ='parent.AddToList(document.forms[0].listAnalyze);']").click()        
        time.sleep(1)
        driver.switch_to.default_content()

        ############################# Get Table ###########################
        parent_window_handle = driver.current_window_handle
        first_window = driver.window_handles[0]

        driver.find_element_by_xpath("//a[@onclick ='getTable(false)']").click()        

        driver.implicitly_wait(30)
        time.sleep(20)

        ############################ POPUP WINDOWs For data ###########################
        handles = driver.window_handles;
        driver.switch_to.window(handles[1])
        print(driver.title)
        print(driver.find_element_by_xpath("//a[@onclick ='saveSettings()']").text)
        print(driver.find_element_by_xpath("//a[@onclick ='execute(false)']").text)

        ############################ aply setting HTML format in popup window ###########################
        driver.implicitly_wait(30)
        time.sleep(1)
        #### Output tab
        driver.find_element_by_xpath("//em[text() = 'Output']").click()
        driver.implicitly_wait(30)
        time.sleep(1)
        driver.find_element_by_xpath("//select[@name='listOutput']/option[text()='HTML Table']").click()
        driver.implicitly_wait(30)
        time.sleep(1)
        ### General tables tab
        driver.find_element_by_xpath("//em[text() = 'General Tables']").click()
        driver.implicitly_wait(30)
        time.sleep(1)
        driver.find_element_by_xpath("//*[@id='SpanColumnPercent']/label").click()
        driver.find_element_by_xpath("//*[@id='SpanColRank']/label").click()
        driver.implicitly_wait(30)
        time.sleep(1)

        ### Apply settings tab
        driver.find_element_by_xpath("//a[text() ='Apply Settings']").click()
        driver.implicitly_wait(30)
        time.sleep(1)

        ###iframe for html data
        iframes_data = driver.find_element_by_xpath("//iframe[contains(@src, 'showData.aspx?guid')]")
        driver.switch_to.frame(iframes_data)
        driver.implicitly_wait(30)

        res = driver.execute_script("return document.documentElement.outerHTML")
        soup = BeautifulSoup(res, 'lxml')

        ################ Put into EXcel##################
        ws = wb.add_sheet(country)
        table = soup.findAll('tbody')[1]  
        rows = table.findAll("tr")
        x = 0
        for tr in rows:
            cols = tr.findAll("td")
            if not cols: 
                # when we hit an empty row, we should not print anything to the workbook
                continue
            y = 0
            for td in cols:
                texte_bu = td.text
                texte_bu = texte_bu.encode('utf-8')
                texte_bu = texte_bu.strip()
                if x <= 4:
                    ws.write(x, y, td.text)
                elif x == 5:    
                    ws.write(x, y + 2, td.text)
                elif x == 6:    
                    ws.write(x, y, td.text)    
                else:
                    ws.write(x, y + 1, td.text) 
                print(x, y, td.text)
                y = y + 1
            # update the row pointer AFTER a row has been printed
            # this avoids the blank row at the top of your table
            x = x + 1

        wb.save('rankings4.xls')
        driver.switch_to.window(handles[0])
        driver.find_element_by_xpath("//a[@href ='chooseSurvey.aspx']").click()
        time.sleep(3)
    except Exception as ex:
        print(ex)

driver.quit()     

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: use options instead of chrome_options


Analyze Data
Save Settings
Apply Settings
0 0 Survey: CASS 2019 (v1), 23035 cases
1 0 Weighted by: weight
2 0 Filter: 20676 cases, expression:((1. Official Target Groups (Also add a Main Field of Study to filter)=Students))
3 0  
4 0  
4 1 2. Main Field of Study
5 0  
5 1 Business
5 2 Engineering/IT
5 3 Natural Sciences
5 4 Liberal Arts/Fine Arts/Education/Social Sciences
5 5 Law
5 6 Health/Medicine
6 0 Ideal employers
6 1 3M
6 2 Column Rank
6 3 106
6 4 62
6 5 58
6 6 112
6 7  
6 8 44
7 0 Accenture
7 1 Column Rank
7 2 66
7 3 84
7 4  
7 5 100
7 6  
7 7 116
8 0 Activision Blizzard
8 1 Column Rank
8 2 81
8 3 48
8 4 66
8 5 53
8 6 88
8 7 94
9 0 Aecon Group
9 1 Column Rank
9 2 136
9 3 46
9 4 91
9 5 137
9 6  
9 7  
10 0 Air Canada
10 1 Column Rank
10 2 7
10 3 13
10 4 18
10 5 13
10 6 8
10 7 11
11 0 Amazon
11 1 Column Rank
11 2 3
11 3 5
11 4 23
11 5 19
11 6 13
11 7 34
12 0 Apple
12 1 Column Rank
12 2 2
12 3 3
12 4 15
12 5 9
12 6 7
12 7 13
13 0 Bain & Company
13 1 Column Rank
13 2 86
13 3 119
13 

105 4 126
105 5 108
105 6 68
105 7 100
106 0 Procter & Gamble (P&G)
106 1 Column Rank
106 2 37
106 3 97
106 4 52
106 5 82
106 6 59
106 7 35
107 0 PwC (PricewaterhouseCoopers)
107 1 Column Rank
107 2 19
107 3 117
107 4 124
107 5 74
107 6 51
107 7 71
108 0 RBC Financial Group / Royal Bank of Canada
108 1 Column Rank
108 2 8
108 3 75
108 4 76
108 5 46
108 6 33
108 7 61
109 0 Riot Games
109 1 Column Rank
109 2 77
109 3 32
109 4 78
109 5 44
109 6 77
109 7 56
110 0 Rogers Communications
110 1 Column Rank
110 2 67
110 3 89
110 4 136
110 5 50
110 6 95
110 7 126
111 0 Royal Canadian Mounted Police (RCMP)
111 1 Column Rank
111 2 61
111 3 68
111 4 22
111 5 12
111 6 3
111 7 19
112 0 Samsung
112 1 Column Rank
112 2 33
112 3 14
112 4 35
112 5 48
112 6 34
112 7 65
113 0 Schlumberger
113 1 Column Rank
113 2 138
113 3 102
113 4 108
113 5 139
113 6  
113 7  
114 0 Schneider Electric
114 1 Column Rank
114 2 139
114 3 88
114 4 130
114 5 118
114 6  
114 7  
115 0 Scotiabank
115 1 Column Rank
115 2 17
115 3

93 7  
94 0 Molson-Coors
94 1 Column Rank
94 2 61
94 3 66
94 4 98
94 5 54
94 6 59
94 7 78
95 0 Monsanto
95 1 Column Rank
95 2 121
95 3 131
95 4 28
95 5 125
95 6  
95 7 74
96 0 Morgan Stanley
96 1 Column Rank
96 2 50
96 3 90
96 4 132
96 5 96
96 6  
96 7 115
97 0 National Bank of Canada
97 1 Column Rank
97 2 29
97 3 123
97 4 92
97 5 50
97 6 44
97 7 63
98 0 Natural Resources Canada
98 1 Column Rank
98 2 77
98 3 37
98 4 2
98 5 26
98 6 29
98 7 24
99 0 Nestlé
99 1 Column Rank
99 2 59
99 3 76
99 4 60
99 5 60
99 6  
99 7 48
100 0 Nissan
100 1 Column Rank
100 2 91
100 3 49
100 4 90
100 5 113
100 6 63
100 7 118
101 0 Ontario Power Generation
101 1 Column Rank
101 2 126
101 3 19
101 4 35
101 5 93
101 6 95
101 7 84
102 0 Ontario Teachers' Pension Plan Board
102 1 Column Rank
102 2 123
102 3 135
102 4 86
102 5 31
102 6 90
102 7 117
103 0 Oracle
103 1 Column Rank
103 2 112
103 3 47
103 4 136
103 5 104
103 6  
103 7 109
104 0 PepsiCo
104 1 Column Rank
104 2 46
104 3 62
104 4 74
104 5 68
104 6 83
104 

97 6 81
97 7  
98 0 Manulife Financial
98 1 Column Rank
98 2 63
98 3 133
98 4 68
98 5 72
98 6 84
98 7 69
99 0 McKinsey & Company
99 1 Column Rank
99 2 17
99 3 47
99 4 83
99 5 51
99 6 106
99 7 85
100 0 Microsoft
100 1 Column Rank
100 2 10
100 3 2
100 4 16
100 5 33
100 6 13
100 7 33
101 0 MNP
101 1 Column Rank
101 2 40
101 3 143
101 4 137
101 5  
101 6  
101 7  
102 0 Molson-Coors
102 1 Column Rank
102 2 41
102 3 50
102 4 56
102 5 56
102 6 51
102 7 53
103 0 Monsanto
103 1 Column Rank
103 2 126
103 3 96
103 4 22
103 5 129
103 6  
103 7 56
104 0 Morgan Stanley
104 1 Column Rank
104 2 51
104 3 105
104 4 126
104 5 77
104 6 50
104 7 104
105 0 National Bank of Canada
105 1 Column Rank
105 2 34
105 3 98
105 4 65
105 5 46
105 6 24
105 7 83
106 0 Natural Resources Canada
106 1 Column Rank
106 2 67
106 3 23
106 4 2
106 5 21
106 6 21
106 7 18
107 0 Nestlé
107 1 Column Rank
107 2 47
107 3 93
107 4 29
107 5 53
107 6 78
107 7 22
108 0 Nexen
108 1 Column Rank
108 2 128
108 3 112
108 4 112
108 5 143
108

...

101 2 118
101 3 62
101 4 50
101 5 107
101 6 94
101 7 102
102 0 Pratt & Whitney Canada
102 1 Column Rank
102 2 109
102 3 37
102 4 125
102 5 102
102 6 62
102 7  
103 0 Procter & Gamble (P&G)
103 1 Column Rank
103 2 16
103 3 51
103 4 41
103 5 42
103 6 73
103 7 42
104 0 Provincial Government
104 1 Column Rank
104 2 20
104 3 27
104 4 5
104 5 3
104 6 5
104 7 8
105 0 PwC (PricewaterhouseCoopers)
105 1 Column Rank
105 2 11
105 3 93
105 4 104
105 5 62
105 6 77
105 7 84
106 0 RBC Financial Group / Royal Bank of Canada
106 1 Column Rank
106 2 12
106 3 92
106 4 78
106 5 38
106 6 24
106 7 48
107 0 RCMP
107 1 Column Rank
107 2 65
107 3 75
107 4 21
107 5 19
107 6 4
107 7 21
108 0 Rio Tinto Alcan
108 1 Column Rank
108 2 117
108 3 52
108 4 71
108 5 99
108 6 92
108 7 106
109 0 Rogers Communications
109 1 Column Rank
109 2 51
109 3 79
109 4 123
109 5 37
109 6 66
109 7 90
110 0 SaskPower
110 1 Column Rank
110 2 122
110 3 91
110 4 102
110 5 130
110 6  
110 7 101
111 0 Schlumberger
111 1 Column Rank
111 2 1

Analyze Data
Save Settings
Apply Settings
0 0 Survey: CASS 2012, 8446 cases
1 0 Weighted by: weight
2 0 Filter: 7234 cases, expression:((Degree for reports=Undergraduate))
3 0  
4 0  
4 1 Main Field of Study
5 0  
5 1 Business
5 2 Engineering/IT
6 0 Ideal employers
6 1 3M
6 2 Column Rank
6 3 87
6 4 55
7 0 Accenture
7 1 Column Rank
7 2 122
7 3 86
8 0 Air Canada
8 1 Column Rank
8 2 8
8 3 11
9 0 AMD Graphics Product Group
9 1 Column Rank
9 2 81
9 3 19
10 0 American Express
10 1 Column Rank
10 2 29
10 3 74
11 0 Apple
11 1 Column Rank
11 2 1
11 3 3
12 0 Bain & Company
12 1 Column Rank
12 2 96
12 3 126
13 0 Bank of America / Merrill Lynch
13 1 Column Rank
13 2 31
13 3 104
14 0 Bank of Canada
14 1 Column Rank
14 2 4
14 3 43
15 0 Barclays
15 1 Column Rank
15 2 74
15 3 101
16 0 BASF
16 1 Column Rank
16 2 124
16 3 103
17 0 Bayer
17 1 Column Rank
17 2 104
17 3 95
18 0 BC Hydro
18 1 Column Rank
18 2 85
18 3 28
19 0 BDO
19 1 Column Rank
19 2 119
19 3 127
20 0 Bell Canada
20 1 Column Rank
20 2 14
20

87 3 95
87 4 27
87 5 16
87 6 31
87 7 23
88 0 Nokia
88 1 Column Rank
88 2 54
88 3 41
88 4 48
88 5  
88 6 50
88 7 79
89 0 Novartis
89 1 Column Rank
89 2 129
89 3 108
89 4 62
89 5  
89 6 120
89 7 84
90 0 Ontario Power Generation
90 1 Column Rank
90 2 90
90 3 27
90 4 64
90 5 45
90 6 91
90 7 56
91 0 Oracle
91 1 Column Rank
91 2 110
91 3 78
91 4 106
91 5 74
91 6 107
91 7 70
92 0 PepsiCo
92 1 Column Rank
92 2 18
92 3 59
92 4 33
92 5 25
92 6 32
92 7 32
93 0 Petro-Canada
93 1 Column Rank
93 2 66
93 3 31
93 4 97
93 5  
93 6 57
93 7 31
94 0 Pfizer
94 1 Column Rank
94 2 101
94 3 88
94 4 12
94 5 111
94 6 54
94 7 16
95 0 Philip Morris International
95 1 Column Rank
95 2 119
95 3 130
95 4  
95 5 65
95 6  
95 7 111
96 0 Philips
96 1 Column Rank
96 2 123
96 3 76
96 4 53
96 5 61
96 6 93
96 7 100
97 0 Pratt & Whitney Canada
97 1 Column Rank
97 2 77
97 3 42
97 4  
97 5 63
97 6 84
97 7 92
98 0 PricewaterhouseCoopers
98 1 Column Rank
98 2 38
98 3 124
98 4 88
98 5 51
98 6 85
98 7 103
99 0 Procter & Gamble
99

133 2 60
133 3 101
133 4 83
133 5  
134 0 The Coca-Cola Company
134 1 Column Rank
134 2 24
134 3 55
134 4 22
134 5 40
135 0 Toronto-Dominion Bank
135 1 Column Rank
135 2 16
135 3 70
135 4 25
135 5 38
136 0 Toyota
136 1 Column Rank
136 2 47
136 3 32
136 4 60
136 5 56
137 0 TransCanada Corporation
137 1 Column Rank
137 2 103
137 3 86
137 4 76
137 5 117
138 0 UBS
138 1 Column Rank
138 2 104
138 3 136
138 4 101
138 5  
139 0 Unilever
139 1 Column Rank
139 2 70
139 3 98
139 4 72
139 5 52
140 0 Wal-Mart
140 1 Column Rank
140 2 97
140 3 96
140 4 53
140 5 30
141 0 WestJet Airlines
141 1 Column Rank
141 2 18
141 3 50
141 4 11
141 5 14
142 0 Weyerhaeuser
142 1 Column Rank
142 2 123
142 3 112
142 4 118
142 5  
143 0 Winners Merchants International
143 1 Column Rank
143 2 64
143 3 113
143 4 38
143 5 60
144 0 Volkswagen
144 1 Column Rank
144 2 62
144 3 63
144 4 43
144 5 113
145 0 Xerox
145 1 Column Rank
145 2 99
145 3 92
145 4 52
145 5 82
Analyze Data
Save Settings
Apply Settings
0 0 Survey: CASS, 

80 0 Honda
80 1 Column Rank
80 2 39
80 3 11
80 4 29
80 5 54
80 6 41
80 7 47
80 8 31
80 9 19
81 0 Honeywell
81 1 Column Rank
81 2 142
81 3 45
81 4 112
81 5 118
81 6 116
81 7  
81 8 125
81 9  
82 0 Hostess Frito-Lay
82 1 Column Rank
82 2 110
82 3 111
82 4 134
82 5 98
82 6 129
82 7  
82 8 95
82 9 75
83 0 HSBC Bank Canada
83 1 Column Rank
83 2 27
83 3 121
83 4 32
83 5 78
83 6 47
83 7 37
83 8 125
83 9  
84 0 Hudson’s Bay Company
84 1 Column Rank
84 2 82
84 3 136
84 4 112
84 5 98
84 6 33
84 7 65
84 8 55
84 9 37
85 0 Husky Energy
85 1 Column Rank
85 2 87
85 3 27
85 4 94
85 5 34
85 6 95
85 7 96
85 8 109
85 9 96
86 0 Hydro One
86 1 Column Rank
86 2 84
86 3 26
86 4 41
86 5 51
86 6 67
86 7 77
86 8 44
86 9 62
87 0 Hydro Quebec
87 1 Column Rank
87 2 77
87 3 29
87 4 43
87 5 24
87 6 29
87 7 16
87 8 84
87 9 25
88 0 IBM
88 1 Column Rank
88 2 25
88 3 6
88 4 3
88 5 36
88 6 37
88 7 23
88 8 46
88 9 25
89 0 Imperial Oil
89 1 Column Rank
89 2 64
89 3 24
89 4 47
89 5 33
89 6 69
89 7 29
89 8 46
89 9 55
90 0 In